# Workshop #2. Expression quantification

## Task 1

Please, provide an example where even EM-based algorithm estimates expression wrong. Describe in which exact cases it might happens.

1. If transcripts are significantly similar (maybe too short or from low-complexity regions)
2. Maybe for some distributions initial abundance estimations may result in different result

In [2]:
import random
import numpy as np

transcripts = [
    "ABCDE",
    "BCDEF",
    "CDEFG",
    "DEFGH",
    "EFGHI"
]

seed = 10
random.seed(seed)

n_molecules = [random.randint(30, 1000) for i in transcripts]
abundance_ground_truth = np.array([n_molecules[i] / sum(n_molecules) for i in range(len(n_molecules))])

frac_reads = 0.1
letters_pool = "".join([transcripts[i] * n_molecules[i] for i in range(len(transcripts))])
reads = random.sample(letters_pool, int(frac_reads * len(letters_pool)))

abundance, counts, step = EM(transcripts, reads)

print("Ground Truth Abundances (%):")
print([f"{abundance_ground_truth[i] * 100:.2f}%" for i in range(len(transcripts))])

print("Estimated Abundances (%):")
print([f"{abundance[i] * 100:.2f}%" for i in range(len(transcripts))])

print("Differences in abundance (in %):")
print((abundance - abundance_ground_truth) * 100)


Ground Truth Abundances (%):
['26.83%', '2.75%', '20.46%', '22.86%', '27.09%']
Estimated Abundances (%):
['27.03%', '0.01%', '19.99%', '24.69%', '28.28%']
Differences in abundance (in %):
[ 0.20069808 -2.73499127 -0.47574171  1.82565445  1.18438045]
